# Convertion of X,Y to Latitude and Longitude

In [9]:
# Using python3.6
from pyproj import Proj, transform
import pandas as pd
import pyproj

In [19]:
events = pd.read_excel("Dataset Exploration\events.xlsx", engine='openpyxl')

In [28]:
events.head(2)

,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y
0,2019,Januari,2019-01-01/03:34:40-1/4,35,Olycka,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 03:34:40,2019-01-01 06:01:30,2019-01-01 03:34:40,2019-01-01 06:15:00,I höjd med Böles-Noran i södergående. En perso...,6774947,606946
1,2019,Januari,2019-01-01/05:17:01-1/4,379,Skadat räcke #,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 05:17:00,2019-01-01 05:18:02,2019-01-01 05:17:01,2019-01-01 16:00:00,I höjd med Böles-Noran i södergående. En perso...,6787923,609936


In [21]:
previous_coords = events[['X', 'Y']]

In [29]:
previous_coords.head(2)

,X,Y
0,6774947,606946
1,6787923,609936


In [32]:
Lat = []
Long = []
inProj = Proj(init='epsg:3006')
outProj = Proj(init='epsg:4326')

i = 0
for X, Y in previous_coords.values:
    x1,y1 = float(X),float(Y)
    longitude,latitude = transform(inProj,outProj,y1, x1)
    
    Lat.append(latitude)
    Long.append(longitude)

events['latitude'] = Lat
events['longitude'] = Long

c:\python36\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\python36\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
c:\python36\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://py

In [34]:
events.head(2)

,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude
0,2019,Januari,2019-01-01/03:34:40-1/4,35,Olycka,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 03:34:40,2019-01-01 06:01:30,2019-01-01 03:34:40,2019-01-01 06:15:00,I höjd med Böles-Noran i södergående. En perso...,6774947,606946,61.094617,16.983451
1,2019,Januari,2019-01-01/05:17:01-1/4,379,Skadat räcke #,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 05:17:00,2019-01-01 05:18:02,2019-01-01 05:17:01,2019-01-01 16:00:00,I höjd med Böles-Noran i södergående. En perso...,6787923,609936,61.210213,17.046390


In [35]:
events.to_csv("events-with-lat-long.csv", sep=",")

# Connecting events to segments

In [50]:
key_segments = pd.read_excel("Dataset Exploration/key segments.xlsx", engine='openpyxl')

In [51]:
key_segmetns.head(2)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1520100057,1071754891,2019-04-16 00:00:00,9999-12-31 00:00:00,930,Syd,"61,165335","16,851284",1,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1520100058,1071754134,2019-04-16 01:00:00,9999-12-31 01:00:00,934,Syd,"61,161885","16,866368",2,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now we want to connect SegmentID with EventID based on the closest distance to event

##### Have to replace , to . for it to be able to convert to float in distance formula...

In [68]:
old_latitudes = key_segments[['Latitud']].values

new_latitudes = []
for old_lat in old_latitudes:
    new_latitudes.append(old_lat[0].replace(",", "."))

    
old_longitudes = key_segments[['Longitud']].values

new_longitudes = []
for old_long in old_longitudes:
    new_longitudes.append(old_long[0].replace(",", "."))

    
key_segments['Latitud'] = new_latitudes
key_segments['Longitud'] = new_longitudes

In [70]:
key_segments.head(1)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1520100057,1071754891,2019-04-16,9999-12-31 00:00:00,930,Syd,61.165335,16.851284,1,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
from geopy.distance import geodesic
import numpy as np

segmentIDs = []
for event_latitude, event_longitude in events[["latitude", "longitude"]].values:
    
    shortest_row_in_key_segments = None
    shortest_distance_in_key_segments = np.inf
    
    for row_index, coords in enumerate(key_segments[["Latitud", "Longitud"]].values):
        
        
        key_segment_latitude = coords[0]
        key_segment_longitude = coords[1]
        

        p1 = (event_latitude,event_longitude) # (lat, lon)
        p2 = (key_segment_latitude,key_segment_longitude) # (lat, lon)
        
        dist = geodesic(p1, p2).meters
        
        if shortest_distance_in_key_segments > dist:
            shortest_distance_in_key_segments = dist
            shortest_row_in_key_segments = row_index
            
    
    segmentIDs.append(int(key_segments.iloc[shortest_row_in_key_segments]["SegmentID"]))
    print("Shortest dist: ", shortest_distance_in_key_segments)
    print("Match Between...")
    print("1. ", key_segments.iloc[shortest_row_in_key_segments]["SegmentID"])
    print("2. Lat: ", event_latitude, " Long: ", event_longitude)
    print("")
            
                     
events['SegmentID'] = segmentIDs
        
        

Shortest dist:  64.4604262184185
Match Between...
1.  1071863723
2. Lat:  61.09461668948084  Long:  16.983451147193456

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  209.85494722257965
Match Between...
1.  428390707
2. Lat:  60.907142867748924  Long:  17.014924840620917

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  369.6864139485644
Match Between...
1.  1071723800
2. Lat:  61.04980449073084  Long:  16.95875417106383

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  195.90475929679386

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  195.90475929679386
Match Between...
1.  428390704
2. Lat:  60.8992478463833  Long:  17.021561639732266

Shortest dist:  420.2850742451857
Match Between...
1.  1071846041
2. Lat:  61.13649118036001  Long:  17.024211776847366

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  420.2850742451857
Match Between...
1.  1071846041
2. Lat:  61.13649118036001  Long:  17.024211776847366

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  362.52776212499083
Match Between...
1.  1071832389
2. Lat:  60.96278116375329  Long:  17.03217011475944

Shortest dist:  218.307681021966

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  96.49422103995015
Match Between...
1.  428403369
2. Lat:  61.241317206089825  Long:  17.033557827100225

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Matc

Shortest dist:  121.80700060480174
Match Between...
1.  375114290
2. Lat:  61.09629586235977  Long:  16.985300122630864

Shortest dist:  420.2850742451857
Match Between...
1.  1071846041
2. Lat:  61.13649118036001  Long:  17.024211776847366

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  10844.695598917375
Match Between...
1.  1071754891
2. Lat:  61.21814549512126  Long:  16.681909103530177

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  218.3076810219666

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  195.84576351318435
Match Between...
1.  428390704
2. Lat:  60.89925654210743  Long:  17.021580627128987

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  8.372485585449661
Match Between...
1.  1071863723
2. Lat:  61.09511152615761  Long:  16.984001550374447

Shortest dist:  195.84576351318435
Match Between...
1.  428390704
2. Lat:  60.89925654210743  Long:  17.021580627128987

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  18.7379733916781

...

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  36.459822126161136
Match Between...
1.  1071723637
2. Lat:  60.997705550529645  Long:  16.994436387566772

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  463.1645142010602
Match Between...
1.  428397753
2. Lat:  60.84145387573889  Long:  17.07641538293439

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  270.0637757714476
Ma

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  420.2850742451857
Match Between...
1.  1071846041
2. Lat:  61.13649118036001  Long:  17.024211776847366

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  468.0873057228949
Match Between...
1.  1071723605
2. Lat:  61.00956541298304  Long:  16.983118144459798

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  365.672714827517
Match Between...
1.  428397748
2. Lat:  60.893951650144274  Long:  17.032120754804993

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  218.30768102196663
Match Between...
1.  1071723605
2. Lat:  61.01172151874142  Long:  16.981846541527528

Shortest dist:  218.30768102196663

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  43.87512159780045
Match Between...
1.  428403355
2. Lat:  61.15825720670352  Long:  17.03400809635149

Shortest dist:  103.19698936440962
Match Between...
1.  428389808
2. Lat:  61.06789503429974  Long:  16.964690557843728

Shortest dist:  144.2040113262129
Match Between...
1.  1071703593
2. Lat:  60.99624453156125  Long:  16.996064598515375

Shortest dist:  123.91654945934516
Match Between...
1.  1071754259
2. Lat:  61.07583430353838  Long:  16.962809619394367

Shortest dist:  349.6041538738807
Match Between...
1.  1071703610
2. Lat:  60.98813275843183  Long:  17.00890409743767

Shortest dist:  293.1297876264947
Match Between...
1.  1071754109
2. Lat:  61.126514495703844  Long:  16.930171711983498

Shortest dist:  308.50306532320053
Match Between...
1.  1071754891
2. Lat:  61.16581548786699  Long:  16.845639449542833

Shortest dist:  293.1297876264947

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  228.83711317520545
Match Between...
1.  1071703530
2. Lat:  60.965284159241534  Long:  17.02996503990528

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  9.299927672782411
Match Between...
1.  1071723711
2. Lat:  60.95964672813704  Long:  17.03455617765656

Shortest dist:  365.672714827517
Match Between...
1.  428397748
2. Lat:  60.893951650144274  Long:  17.032120754804993

Shortest dist:  13.604072681760393
Match Between...
1.  1071727530
2. Lat:  60.9099000297084  Long:  17.01432430535885

Shortest dist:  13.604072681760393
Mat

Shortest dist:  104.5952423755531
Match Between...
1.  428403354
2. Lat:  61.21021263108927  Long:  17.046389852481724

Shortest dist:  195.90475929679386
Match Between...
1.  428390704
2. Lat:  60.8992478463833  Long:  17.021561639732266

Shortest dist:  195.90475929679386
Match Between...
1.  428390704
2. Lat:  60.8992478463833  Long:  17.021561639732266



In [80]:
events.head(5)

,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
0,2019,Januari,2019-01-01/03:34:40-1/4,35,Olycka,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 03:34:40,2019-01-01 06:01:30,2019-01-01 03:34:40,2019-01-01 06:15:00,I höjd med Böles-Noran i södergående. En perso...,6774947,606946,61.094617,16.983451,1071863723
1,2019,Januari,2019-01-01/05:17:01-1/4,379,Skadat räcke #,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 05:17:00,2019-01-01 05:18:02,2019-01-01 05:17:01,2019-01-01 16:00:00,I höjd med Böles-Noran i södergående. En perso...,6787923,609936,61.210213,17.046390,428403354
2,2019,Januari,2019-01-01/13:25:07-1/4,35,Olycka,E4 från Trafikplats Hagsta (201) till Axmartav...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2019-01-01 13:25:06,2019-01-01 13:47:34,2019-01-01 13:24:52,2019-01-01 14:30:00,"Singelolycka, personbil kört i diket strax nor...",6754124,609285,60.907143,17.014925,428390707
3,2019,Januari,2019-01-01/13:41:56-1/4,35,Olycka,E4 från Axmartavlan (202) till Trafikplats Tön...,E4,Båda,OOOOOOOOO|OOOOOOOOO,2019-01-01 13:41:56,2019-01-01 13:42:39,2019-01-01 13:41:56,NaN,"Singelolycka, personbil kört i diket strax nor...",6765714,607139,61.011722,16.981847,1071723605
4,2019,Januari,2019-01-01/14:13:46-1/4,32,Stillastående fordon,E4 från Axmartavlan (202) till Trafikplats Tön...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2019-01-01 14:13:46,2019-01-01 15:21:42,2019-01-01 14:13:46,2019-01-01 15:30:00,En stillastående personbil intill vägen strax ...,6769917,605764,61.049804,16.958754,1071723800


In [81]:
events.to_csv("events-with-lat-long-segmentID.csv", sep=",")